In [11]:
import os
import numpy as np
import pandas as pd
import mne 


# fifFileMddPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16channels/mdd1624.fif"
# fifFileHcPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16channels/hc1624.fif"

fifFileHcEpochPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif"
fifFileMddEpochPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif"   

allFeatureDirPath = "/Users/wachiii/Workschii/brain-mdd/data/balancedfeatures"
# mddRaw = mne.io.read_raw_fif(fifFileMddPath)
# hcRaw = mne.io.read_raw_fif(fifFileHcPath)
# print(mddRaw)
# print(hcRaw)

## Preprocessing
- bandpass filter 0.5 - 40 Hz
- Epoching for 10 sec => 10*250 = 2500 samples

In [ ]:
# do preprocessing: bandpass filter 0.5 - 40 Hz
mddEeg = mddRaw.load_data()
hcEeg = hcRaw.load_data()

mddEeg.filter(0.5, 40, fir_design='firwin')
hcEeg.filter(0.5, 40, fir_design='firwin')

# plot psd only 1 channel
mddEeg.plot_psd(fmin=0.5, fmax=40, average=True, spatial_colors=True) 
hcEeg.plot_psd(fmin=0.5, fmax=40, average=True, spatial_colors=True)

# do epoching; fs = 250, total samples = 1784136 where 1784136/24 = 74339.0 samples per subject -> i want epoch for 10 seconds
# 10 seconds = 2500 samples

mddEpochs = mne.make_fixed_length_epochs(mddEeg, duration=10, preload=True)
hcEpochs = mne.make_fixed_length_epochs(hcEeg, duration=10, preload=True)

print(mddEpochs)
print(hcEpochs)

# mddEpochs.save("/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif", overwrite=True)
# hcEpochs.save("/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif", overwrite=True)

In [ ]:
74339.0/(10*250)

## Feature Extraction

### 1. Band Power - PSD

In [53]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.psd import EEGPowerSpectrum
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "norm_power": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    }    

In [54]:
# Load the epochs
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)

print(len(mddEpochs))
print(len(hcEpochs))

print(mddEpochs.get_data()[0].shape)
print(hcEpochs.get_data()[0].shape)

Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/3406999483.py:2: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  mddEpochs = mne.read_epochs(fifFileMddEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/3406999483.py:3: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  hcEpochs = mne.read_epochs(fifFileHcEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
713
713
(16, 2500)
(16, 2500)


In [55]:
psdObjMdd = EEGPowerSpectrum(mddEpochs, 250, ifNormalize=False)
mddPsdFeatures = psdObjMdd.run()
psdObjCtrl = EEGPowerSpectrum(hcEpochs, 250, ifNormalize=False)
hcPsdFeatures = psdObjCtrl.run()
print(mddPsdFeatures.shape)
print(hcPsdFeatures.shape)

(713, 5, 16)
(713, 5, 16)


In [57]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/bandpower/mddBandpowerFeatures.npy", mddPsdFeatures)
np.save(allFeatureDirPath + "/bandpower/hcBandpowerFeatures.npy", hcPsdFeatures)

### 2. Relative Power

In [ ]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.psd import EEGPowerSpectrum
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "norm_power": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    }    

In [51]:
# Load the epochs
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)

print(len(mddEpochs))
print(len(hcEpochs))

print(mddEpochs.get_data()[0].shape)
print(hcEpochs.get_data()[0].shape)

Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/3406999483.py:2: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  mddEpochs = mne.read_epochs(fifFileMddEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available
Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
713
713


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/3406999483.py:3: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  hcEpochs = mne.read_epochs(fifFileHcEpochPath)


(16, 2500)
(16, 2500)


In [58]:
psdObjMdd = EEGPowerSpectrum(mddEpochs, 250, ifNormalize=True)
mddPsdFeatures = psdObjMdd.run()
psdObjCtrl = EEGPowerSpectrum(hcEpochs, 250, ifNormalize=True)
hcPsdFeatures = psdObjCtrl.run()
print(mddPsdFeatures.shape)
print(hcPsdFeatures.shape)

(713, 5, 16)
(713, 5, 16)


In [59]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/relativepower/mddRelativepowerFeatures.npy", mddPsdFeatures)
np.save(allFeatureDirPath + "/relativepower/hcRelativepowerFeatures.npy", hcPsdFeatures)

### 3. HFD

In [ ]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.hfd import EEGHiguchiFractalDimension
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "HFD": {
        "mdd": [],
        "control": []
    }
}

In [22]:
# Load the epochs
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)
mddEpochs = mddEpochs.get_data()
hcEpochs = hcEpochs.get_data()
print(mddEpochs.shape)
print(hcEpochs.shape)

Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/1695134047.py:2: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  mddEpochs = mne.read_epochs(fifFileMddEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available
Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/1695134047.py:3: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  hcEpochs = mne.read_epochs(fifFileHcEpochPath)


(713, 16, 2500)
(713, 16, 2500)


In [26]:
hfdObjMdd = EEGHiguchiFractalDimension(mddEpochs)
mddHfdFeatures = hfdObjMdd.run()
hfdObjControl = EEGHiguchiFractalDimension(hcEpochs)
hcHfdFeatures = hfdObjControl.run()

print(mddHfdFeatures.shape)
print(hcHfdFeatures.shape)

(713, 16)
(713, 16)


In [28]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/hfd/mddHfdFeatures.npy", mddHfdFeatures)
np.save(allFeatureDirPath + "/hfd/hcHfdFeatures.npy", hcHfdFeatures)

### 4. Coherences

In [61]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.coherence import EEGCoherence
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9 
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "coherence": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    } 


In [64]:
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
mddEpochs = mddEpochs.get_data()
# print paired channels
coherenceObjMdd = EEGCoherence(mddEpochs, MODMA_SFREQ)
print(coherenceObjMdd.channel_pairs)

Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/752862024.py:1: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  mddEpochs = mne.read_epochs(fifFileMddEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (6, 15), (7, 8), (7, 9), (7, 10), (7, 11), (7, 12), (7, 13), (7, 14), (7, 15), (8, 9), (8, 10), (8, 11), (8, 12), (8, 13), (8, 14), (8, 15), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (

In [30]:
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)
mddEpochs = mddEpochs.get_data()
hcEpochs = hcEpochs.get_data()
print(mddEpochs.shape)
print(hcEpochs.shape)

Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/1994406543.py:1: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  mddEpochs = mne.read_epochs(fifFileMddEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available
Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/1994406543.py:2: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  hcEpochs = mne.read_epochs(fifFileHcEpochPath)


(713, 16, 2500)
(713, 16, 2500)


In [31]:
coherenceObjMdd = EEGCoherence(mddEpochs, MODMA_SFREQ)
mddCohFeatures = coherenceObjMdd.run()
coherenceObjControl = EEGCoherence(hcEpochs, MODMA_SFREQ)
hcCohFeatures = coherenceObjControl.run()

print(mddCohFeatures.shape)
print(hcCohFeatures.shape)

(713, 5, 120)
(713, 5, 120)


In [32]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/coh/mddCohFeatures.npy", mddCohFeatures)
np.save(allFeatureDirPath + "/coh/hcCohFeatures.npy", hcCohFeatures)

### 5. Cd

In [34]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.correlationdimension import EEGCorrelationDimension
import mne
import enum

dotenv.load_dotenv()

class ChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "CD": {
        "mdd": [],
        "control": []
    }
}

In [35]:
mddEpochs = mne.read_epochs(fifFileMddEpochPath)
hcEpochs = mne.read_epochs(fifFileHcEpochPath)
mddEpochs = mddEpochs.get_data()
hcEpochs = hcEpochs.get_data()
print(mddEpochs.shape)
print(hcEpochs.shape)

Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/1994406543.py:1: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/mdd1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  mddEpochs = mne.read_epochs(fifFileMddEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_16757/1994406543.py:2: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/balanced16preprocess/hc1624preprocessed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  hcEpochs = mne.read_epochs(fifFileHcEpochPath)


Not setting metadata
713 matching events found
No baseline correction applied
0 projection items activated
(713, 16, 2500)
(713, 16, 2500)


In [36]:
cdObjMdd = EEGCorrelationDimension(mddEpochs)
mddCdFeatures = cdObjMdd.run()
cdObjControl = EEGCorrelationDimension(hcEpochs)
hcCdFeatures = cdObjControl.run()

for epoch in mddCdFeatures:
    featureDict["CD"]["mdd"].append(epoch)
for epoch in hcCdFeatures:
    featureDict["CD"]["control"].append(epoch)

In [37]:
# save at allFeatureDirPath + /psd
np.save(allFeatureDirPath + "/cd/mddCdFeatures.npy", mddCdFeatures)
np.save(allFeatureDirPath + "/cd/hcCdFeatures.npy", hcCdFeatures)

## Feature Selection & ML

In [73]:
import os
import numpy as np
import joblib
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


allFeatureDirPath = "/Users/wachiii/Workschii/brain-mdd/data/balancedfeatures"

hcPsdFeatures = np.load(allFeatureDirPath + "/bandpower/hcBandpowerFeatures.npy")
mddPsdFeatures = np.load(allFeatureDirPath + "/bandpower/mddBandpowerFeatures.npy")
hcRelativepowerFeatures = np.load(allFeatureDirPath + "/relativepower/hcRelativepowerFeatures.npy")
mddRelativepowerFeatures = np.load(allFeatureDirPath + "/relativepower/mddRelativepowerFeatures.npy")
hcCohFeatures = np.load(allFeatureDirPath + "/coh/hcCohFeatures.npy")
mddCohFeatures = np.load(allFeatureDirPath + "/coh/mddCohFeatures.npy")
hcHfdFeatures = np.load(allFeatureDirPath + "/hfd/hcHfdFeatures.npy")
mddHfdFeatures = np.load(allFeatureDirPath + "/hfd/mddHfdFeatures.npy")
hcCdFeatures = np.load(allFeatureDirPath + "/cd/hcCdFeatures.npy")
mddCdFeatures = np.load(allFeatureDirPath + "/cd/mddCdFeatures.npy")

print(f"hc psd feature shape: {hcPsdFeatures.shape}")
print(f"mdd psd feature shape: {mddPsdFeatures.shape}")
print(f"relative power hc feature shape: {hcRelativepowerFeatures.shape}")
print(f"relative power mdd feature shape: {mddRelativepowerFeatures.shape}")
print(f"hc coh feature shape: {hcCohFeatures.shape}")
print(f"mdd coh feature shape: {mddCohFeatures.shape}")
print(f"hc hfd feature shape: {hcHfdFeatures.shape}")
print(f"mdd hfd feature shape: {mddHfdFeatures.shape}")
print(f"hc cd feature shape: {hcCdFeatures.shape}")
print(f"mdd cd feature shape: {mddCdFeatures.shape}")

hc psd feature shape: (713, 5, 16)
mdd psd feature shape: (713, 5, 16)
relative power hc feature shape: (713, 5, 16)
relative power mdd feature shape: (713, 5, 16)
hc coh feature shape: (713, 5, 120)
mdd coh feature shape: (713, 5, 120)
hc hfd feature shape: (713, 16)
mdd hfd feature shape: (713, 16)
hc cd feature shape: (713, 16)
mdd cd feature shape: (713, 16)


In [76]:
# Flatten features to make the shape consistent
hcPsdFeatures = hcPsdFeatures.reshape(hcPsdFeatures.shape[0], -1)  # Flatten to (samples, features)
hcRelativepowerFeatures = hcRelativepowerFeatures.reshape(hcRelativepowerFeatures.shape[0], -1)
hcCohFeatures = hcCohFeatures.reshape(hcCohFeatures.shape[0], -1)
hcHfdFeatures = hcHfdFeatures.reshape(hcHfdFeatures.shape[0], -1)
hcCdFeatures = hcCdFeatures.reshape(hcCdFeatures.shape[0], -1)

mddPsdFeatures = mddPsdFeatures.reshape(mddPsdFeatures.shape[0], -1)
mddRelativepowerFeatures = mddRelativepowerFeatures.reshape(mddRelativepowerFeatures.shape[0], -1)
mddCohFeatures = mddCohFeatures.reshape(mddCohFeatures.shape[0], -1)
mddHfdFeatures = mddHfdFeatures.reshape(mddHfdFeatures.shape[0], -1)
mddCdFeatures = mddCdFeatures.reshape(mddCdFeatures.shape[0], -1)

print(f"hc psd feature shape: {hcPsdFeatures.shape}")
print(f"mdd psd feature shape: {mddPsdFeatures.shape}")
print(f"relative power hc feature shape: {hcRelativepowerFeatures.shape}")
print(f"relative power mdd feature shape: {mddRelativepowerFeatures.shape}")
print(f"hc coh feature shape: {hcCohFeatures.shape}")
print(f"mdd coh feature shape: {mddCohFeatures.shape}")
print(f"hc hfd feature shape: {hcHfdFeatures.shape}")
print(f"mdd hfd feature shape: {mddHfdFeatures.shape}")
print(f"hc cd feature shape: {hcCdFeatures.shape}")
print(f"mdd cd feature shape: {mddCdFeatures.shape}")

hc psd feature shape: (713, 80)
mdd psd feature shape: (713, 80)
relative power hc feature shape: (713, 80)
relative power mdd feature shape: (713, 80)
hc coh feature shape: (713, 600)
mdd coh feature shape: (713, 600)
hc hfd feature shape: (713, 16)
mdd hfd feature shape: (713, 16)
hc cd feature shape: (713, 16)
mdd cd feature shape: (713, 16)


In [74]:
models = {
    "KNN": (KNeighborsClassifier(), {"n_neighbors": [3, 5, 7, 9], "weights": ["uniform", "distance"]}),
    "SVM": (SVC(), {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]}),
    "Decision Tree": (DecisionTreeClassifier(), {"max_depth": [5, 10, 15]}),
    "Random Forest": (RandomForestClassifier(), {"n_estimators": [50, 100], "max_depth": [10, 20]}),
    "Logistic Regression": (LogisticRegression(), {"C": [0.01, 0.1, 1]})
}

featureSelectors = {
    "NoFeatureSelection": None,
    "SelectKBest": SelectKBest(score_func=f_classif, k=100),
    "VarianceThreshold": VarianceThreshold(threshold=0.01),  # Removes low variance features
}

def get_stratified_kfold_data(X, y, nSplits=5):
    skf = StratifiedKFold(n_splits=nSplits, shuffle=True, random_state=42)
    return skf.split(X, y)

In [77]:
X = np.concatenate([hcPsdFeatures, hcRelativepowerFeatures, hcCohFeatures, hcHfdFeatures, hcCdFeatures], axis=1)
y = np.concatenate([np.zeros(hcPsdFeatures.shape[0]), np.ones(mddPsdFeatures.shape[0])])

In [ ]:
results = []

for modelName, (model, paramGrid) in models.items():
    for featureSelectorName, featureSelector in featureSelectors.items():
        print(f"Training {modelName} with {featureSelectorName}...")
        
        if featureSelector is not None:
            selector = featureSelector
        else:
            selector = None
        
        foldAccuracies = []
        foldPrecisions = []
        foldRecalls = []
        foldF1Scores = []
        selectedFeatures = []
        foldHcTrain = []
        foldMddTrain = []
        foldHcVal = []
        foldMddVal = []
        foldHcTest = []
        foldMddTest = []

        for trainIdx, valIdx in get_stratified_kfold_data(X, y, nSplits=5):
            XTrain, XVal = X[trainIdx], X[valIdx]
            yTrain, yVal = y[trainIdx], y[valIdx]
            
            hcTrain = np.sum(yTrain == 0)
            mddTrain = np.sum(yTrain == 1)
            hcVal = np.sum(yVal == 0)
            mddVal = np.sum(yVal == 1)
            hcTest = np.sum(y == 0) - hcTrain - hcVal
            mddTest = np.sum(y == 1) - mddTrain - mddVal
            
            foldHcTrain.append(hcTrain)
            foldMddTrain.append(mddTrain)
            foldHcVal.append(hcVal)
            foldMddVal.append(mddVal)
            foldHcTest.append(hcTest)
            foldMddTest.append(mddTest)
            
            if selector is not None:
                XTrain = selector.fit_transform(XTrain, yTrain)
                XVal = XVal[:, selector.get_support()]
            
            model.fit(XTrain, yTrain)
            
            yPred = model.predict(XVal)
            foldAccuracies.append(accuracy_score(yVal, yPred))
            foldPrecisions.append(precision_score(yVal, yPred))
            foldRecalls.append(recall_score(yVal, yPred))
            foldF1Scores.append(f1_score(yVal, yPred))
            
            if selector is not None:
                selectedFeatures.append(np.where(selector.get_support())[0])

        avgAccuracy = np.mean(foldAccuracies)
        avgPrecision = np.mean(foldPrecisions)
        avgRecall = np.mean(foldRecalls)
        avgF1Score = np.mean(foldF1Scores)
        
        selectedFeatures = np.array(selectedFeatures)
        significantFeatures = np.unique(selectedFeatures)  

        results.append({
            "model": modelName,
            "feature_selection": featureSelectorName,
            "avg_accuracy": avgAccuracy,
            "avg_precision": avgPrecision,
            "avg_recall": avgRecall,
            "avg_f1_score": avgF1Score,
            "significant_features": significantFeatures,
            "hc_train": np.mean(foldHcTrain),
            "mdd_train": np.mean(foldMddTrain),
            "hc_val": np.mean(foldHcVal),
            "mdd_val": np.mean(foldMddVal),
            "hc_test": np.mean(foldHcTest),
            "mdd_test": np.mean(foldMddTest)
        })
        
        print(f"Avg Accuracy: {avgAccuracy:.4f}, Avg Precision: {avgPrecision:.4f}, "
              f"Avg Recall: {avgRecall:.4f}, Avg F1 Score: {avgF1Score:.4f}")
        print(f"Significant Features: {significantFeatures[:10]}...") 
        
        modelFilename = f"/Users/wachiii/Workschii/brain-mdd/models/feature_selection/{modelName}_{featureSelectorName}.pkl"
        joblib.dump(model, modelFilename)
        print(f"Saved {modelName} with {featureSelectorName} to {modelFilename}")

resultsDf = pd.DataFrame(results)
resultsDf.to_csv("model_performance_results.csv", index=False)
resultsDf.to_excel("model_performance_results.xlsx", index=False)

print("Finished saving results and models!")